In [357]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers import LSTM

In [358]:
x_train = pd.read_csv('./trajectory/traj_x_train.csv', header=None)
x_train = np.array(x_train)
x_train = x_train.reshape(210,3)

In [359]:
x_max = max(x_train[:,0])
x_min = min(x_train[:,0])
y_max = max(x_train[:,1])
y_min = min(x_train[:,1])
z_max = max(x_train[:,2])
z_min = min(x_train[:,2])
print(x_min, x_max, y_min, y_max, z_min, z_max)

66.4967 95.4134 -15.1893 281.438 1.43447 58.4673


In [360]:
from keras.utils import np_utils

In [361]:
y_train = pd.read_csv('./trajectory/traj_y_train.csv', header=None)
#y_train = np.array([1,1,2,2,3,3])
#y_train = y_train.reshape(6,1)
#y_train

In [362]:
#y_train = np_utils.to_categorical(y_train, 3)
y_train

,0,1,2
0,1,0,0
1,1,0,0
2,0,1,0
3,0,1,0
4,0,0,1
5,0,0,1


In [363]:
#x_train = np.array(x_train)
x_train = x_train.reshape(6, 35, 3)
x_train

array([[[ 8.00810e+01,  2.78725e+02,  5.15481e+01],
        [ 7.98090e+01,  2.68847e+02,  5.30291e+01],
        [ 7.95380e+01,  2.57158e+02,  5.47825e+01],
        [ 7.92844e+01,  2.45890e+02,  5.59122e+01],
        [ 7.88713e+01,  2.39182e+02,  5.60064e+01],
        [ 7.84739e+01,  2.32452e+02,  5.55936e+01],
        [ 7.82590e+01,  2.18466e+02,  5.65888e+01],
        [ 7.78914e+01,  2.08481e+02,  5.59948e+01],
        [ 7.75442e+01,  2.00509e+02,  5.48337e+01],
        [ 7.73829e+01,  1.89691e+02,  5.43085e+01],
        [ 7.70679e+01,  1.80679e+02,  5.29485e+01],
        [ 7.67701e+01,  1.71867e+02,  5.12320e+01],
        [ 7.66480e+01,  1.62067e+02,  4.97529e+01],
        [ 7.65374e+01,  1.52249e+02,  4.72838e+01],
        [ 7.62805e+01,  1.44166e+02,  4.47854e+01],
        [ 7.58809e+01,  1.38212e+02,  4.08821e+01],
        [ 7.58120e+01,  1.28027e+02,  3.82420e+01],
        [ 7.55966e+01,  1.20273e+02,  3.48816e+01],
        [ 7.53965e+01,  1.12388e+02,  3.08262e+01],
        [ 7.

In [364]:
y_train = np.array(y_train)
#y_train.reshape(6, 3)
y_train.shape

(6, 3)

In [365]:
tmp = np.zeros((6,35,3))
for i in range(x_train.shape[0]):
    for j in range(x_train.shape[1]):
        tmp[i,j,0] = (x_train[i,j,0]-x_min)/(x_max-x_min)
        tmp[i,j,1] = (x_train[i,j,1]-y_min)/(y_max-y_min)
        tmp[i,j,2] = (x_train[i,j,2]-z_min)/(z_max-z_min)
        
tmp

array([[[0.46977352, 0.99085384, 0.8786804 ],
        [0.46036719, 0.95755279, 0.9046479 ],
        [0.45099545, 0.91814644, 0.9353916 ],
        [0.44222543, 0.88015938, 0.95519949],
        [0.42793956, 0.85754514, 0.95685117],
        [0.41419664, 0.83485674, 0.94961323],
        [0.40676495, 0.78770666, 0.96706283],
        [0.39405257, 0.75404489, 0.95664778],
        [0.38204567, 0.72716941, 0.93628933],
        [0.37646758, 0.69069941, 0.9270806 ],
        [0.36557422, 0.66031785, 0.90323468],
        [0.35527567, 0.63061053, 0.87313798],
        [0.3510532 , 0.59757244, 0.84720379],
        [0.34722842, 0.56447367, 0.80391119],
        [0.33834428, 0.53722398, 0.76010484],
        [0.32452527, 0.51715166, 0.69166531],
        [0.32214257, 0.48281564, 0.64537443],
        [0.31469359, 0.45667509, 0.58645398],
        [0.30777371, 0.43009291, 0.51534756],
        [0.30159389, 0.40103288, 0.44926107],
        [0.30072588, 0.36927417, 0.3805936 ],
        [0.29531032, 0.34212225, 0

In [366]:
n_hidden = 512
n_step = tmp.shape[1]
n_input = tmp.shape[2]
n_classes = 3
batch_size = 2
training_iters = 150

In [367]:
model = Sequential()
model.add(LSTM(units=n_hidden, batch_size=(None, n_step, n_input), unroll=True))
model.add(Dense(units=n_classes, activation='softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [368]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=training_iters)

result = model.evaluate(x_train, y_train, batch_size=batch_size)
print ('\nTraining accuracy:', result[1])

Epoch 1/150
6/6 [==============================] - 16s 3s/step - loss: 0.2934 - acc: 0.1667 
Epoch 2/150
6/6 [==============================] - 0s 10ms/step - loss: 0.2269 - acc: 0.3333
Epoch 3/150
6/6 [==============================] - 0s 9ms/step - loss: 0.1660 - acc: 0.8333
Epoch 4/150
6/6 [==============================] - 0s 9ms/step - loss: 0.1173 - acc: 1.0000
Epoch 5/150
6/6 [==============================] - 0s 11ms/step - loss: 0.0854 - acc: 1.0000
Epoch 6/150
6/6 [==============================] - 0s 10ms/step - loss: 0.0772 - acc: 1.0000
Epoch 7/150
6/6 [==============================] - 0s 9ms/step - loss: 0.0334 - acc: 1.0000
Epoch 8/150
6/6 [==============================] - 0s 10ms/step - loss: 0.0320 - acc: 1.0000
Epoch 9/150
6/6 [==============================] - 0s 10ms/step - loss: 0.0100 - acc: 1.0000
Epoch 10/150
6/6 [==============================] - 0s 10ms/step - loss: 0.0076 - acc: 1.0000
Epoch 11/150
6/6 [==============================] - 0s 10ms/step - loss:

6/6 [==============================] - 0s 9ms/step - loss: 2.1653e-05 - acc: 1.0000
Epoch 86/150
6/6 [==============================] - 0s 10ms/step - loss: 2.1243e-05 - acc: 1.0000
Epoch 87/150
6/6 [==============================] - 0s 10ms/step - loss: 2.0936e-05 - acc: 1.0000
Epoch 88/150
6/6 [==============================] - 0s 9ms/step - loss: 2.0595e-05 - acc: 1.0000
Epoch 89/150
6/6 [==============================] - 0s 10ms/step - loss: 2.0271e-05 - acc: 1.0000
Epoch 90/150
6/6 [==============================] - 0s 11ms/step - loss: 1.9964e-05 - acc: 1.0000
Epoch 91/150
6/6 [==============================] - 0s 13ms/step - loss: 1.9609e-05 - acc: 1.0000
Epoch 92/150
6/6 [==============================] - 0s 12ms/step - loss: 1.9437e-05 - acc: 1.0000
Epoch 93/150
6/6 [==============================] - 0s 11ms/step - loss: 1.9098e-05 - acc: 1.0000
Epoch 94/150
6/6 [==============================] - 0s 10ms/step - loss: 1.8777e-05 - acc: 1.0000
Epoch 95/150
6/6 [=================

In [369]:
x_test = pd.read_csv('./trajectory/traj_x_test.csv', header=None)
x_test = np.array(x_test)
x_test.shape

(3, 105)

In [370]:
x_test = x_test.reshape(3, 35, 3)
test_up = x_test[0, :, :]
test_right = x_test[1, :, :]
test_down = x_test[2, :, :]
test_down.shape

(35, 3)

In [371]:
test_up = test_up.reshape(1, 35, 3)
test_right = test_right.reshape(1, 35, 3)
test_down = test_down.reshape(1, 35, 3)

In [372]:
nor_test_up = np.copy(test_up)
nor_test_right = np.copy(test_right)
nor_test_down = np.copy(test_down)

In [373]:
for j in range(35):
    nor_test_up[:,j,0] = (nor_test_up[:,j,0]-x_min)/(x_max-x_min)
    nor_test_up[:,j,1] = (nor_test_up[:,j,1]-y_min)/(y_max-y_min)
    nor_test_up[:,j,2] = (nor_test_up[:,j,2]-z_min)/(z_max-z_min)
    
    nor_test_right[:,j,0] = (nor_test_right[:,j,0]-x_min)/(x_max-x_min)
    nor_test_right[:,j,1] = (nor_test_right[:,j,1]-y_min)/(y_max-y_min)
    nor_test_right[:,j,2] = (nor_test_right[:,j,2]-z_min)/(z_max-z_min)
        
    nor_test_down[:,j,0] = (nor_test_down[:,j,0]-x_min)/(x_max-x_min)
    nor_test_down[:,j,1] = (nor_test_down[:,j,1]-y_min)/(y_max-y_min)
    nor_test_down[:,j,2] = (nor_test_down[:,j,2]-z_min)/(z_max-z_min)

In [374]:
nor_test_up

array([[[0.48321212, 0.97410555, 0.89538832],
        [0.480698  , 0.94156977, 0.92071759],
        [0.47832567, 0.91032181, 0.94169323],
        [0.47555911, 0.87249319, 0.95822231],
        [0.47349456, 0.84328482, 0.96542693],
        [0.47154067, 0.81510131, 0.96941937],
        [0.46926171, 0.78087654, 0.96715927],
        [0.46668188, 0.74213769, 0.96464317],
        [0.45965134, 0.72144169, 0.93914908],
        [0.45249285, 0.69573603, 0.91698466],
        [0.45600293, 0.65545653, 0.90596118],
        [0.45458507, 0.62641672, 0.87828589],
        [0.44310035, 0.60720406, 0.83677822],
        [0.44195223, 0.57390301, 0.79554232],
        [0.44097356, 0.54144477, 0.74731045],
        [0.43533322, 0.51416137, 0.69400957],
        [0.43493898, 0.48760954, 0.63859763],
        [0.43424388, 0.45442985, 0.58835464],
        [0.42967904, 0.4317819 , 0.51540718],
        [0.42532516, 0.40980146, 0.44494601],
        [0.42094707, 0.37765742, 0.37527386],
        [0.42154879, 0.35282693, 0

In [375]:
pred_up = model.predict(nor_test_up)
pred_right = model.predict(nor_test_right)
pred_down = model.predict(nor_test_down)

In [376]:
print(pred_up)
print(pred_right)
print(pred_down)

[[0.42767993 0.20483379 0.3674863 ]]
[[0.427204  0.2001499 0.3726461]]
[[0.37451854 0.27134293 0.3541385 ]]
